In [1]:
import cartopy.crs as ccrs
import cv2
import matplotlib.pyplot as plt
import numpy as np

from src.Dataset.create_data import DataBuilder
from src.Dataset.fuxi_dataset import FuXiDataset

In [2]:
start_year = 1958
end_year = 1958
data_dir = "/Users/ksoll/git/FuXiClimatePrediction/data"

builder = DataBuilder(data_dir, start_year, end_year)
builder.generate_data()

In [3]:
ds = FuXiDataset(
    dataset_path='/Users/ksoll/git/FuXiClimatePrediction/data/1958_1958.zarr',
    means_file='/Users/ksoll/git/FuXiClimatePrediction/data/mean_1958_1958.zarr'
)

In [4]:
fps = 20
frame_size = (1920, 1080)

In [5]:
def get_variable_data():
    data = np.zeros((len(ds), 35, 121, 240))
    for idx, item in enumerate(iter(ds)):
        data[idx, :, :, :] = item[0][0, :, :, :]
    return data

In [6]:
def plot_data(data, var_idx, time_idx):
    data = data[time_idx][var_idx]
    fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.coastlines()
    lons = np.linspace(-180, 180, data.shape[1])
    lats = np.linspace(-90, 90, data.shape[0])
    im = ax.pcolormesh(lons, lats, data, transform=ccrs.PlateCarree(), shading='auto')
    plt.colorbar(im, ax=ax, orientation='vertical')
    name, level = ds.get_var_name_and_level_at_idx(var_idx)
    ax.set_title(f'Var: {name} at Level: {level} at Time idx: {time_idx}')
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    data = cv2.cvtColor(data, cv2.COLOR_RGBA2BGR)
    plt.close(fig)
    return data

In [7]:
path = f"/Users/ksoll/git/FuXiClimatePrediction/data-viz/all.mp4"
out = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size)
data = get_variable_data()
for var_idx in range(35):
    for time_idx in range(len(ds)):
        im = plot_data(data, var_idx, time_idx)
        bild_resized = cv2.resize(im, frame_size)
        out.write(bild_resized)
out.release()